In [21]:
import pandas as pd
# imports googlesearch to use requests and BS4 to scrape google
from googlesearch import search


In [3]:
# In the future, we can improve this by being able to read in any/all the datasets

# reads in the data set
#data = pd.read_csv(r"Affiliation_Parsing/NeurIPS affiliations/NeurIPSaffiliations-2010-affiliation.csv")
data = pd.read_csv(r"Affiliation_Parsing/NeurIPS affiliations gender/NeurIPS-2010/xaa.csv")
data.info()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 19 entries, 0 to 18
Data columns (total 7 columns):
 #   Column                    Non-Null Count  Dtype 
---  ------                    --------------  ----- 
 0   Unnamed: 0                19 non-null     int64 
 1   Unnamed: 0.1              19 non-null     int64 
 2   first-author              19 non-null     object
 3   first-author-affiliation  19 non-null     object
 4   last-author               19 non-null     object
 5   last-author-affiliation   19 non-null     object
 6   title                     19 non-null     object
dtypes: int64(2), object(5)
memory usage: 1.2+ KB


In [4]:
# drops every column that is not the authors or their affiliations
data = data.drop(data.columns[0:2], axis=1)
data = data.drop(data.columns[4:5], axis=1)
data.info()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 19 entries, 0 to 18
Data columns (total 4 columns):
 #   Column                    Non-Null Count  Dtype 
---  ------                    --------------  ----- 
 0   first-author              19 non-null     object
 1   first-author-affiliation  19 non-null     object
 2   last-author               19 non-null     object
 3   last-author-affiliation   19 non-null     object
dtypes: object(4)
memory usage: 736.0+ bytes


In [5]:
# imports BS4 libraries for webscraping
from re import I
import requests
from bs4 import BeautifulSoup


In [6]:
# Need to add in they/them
# Improve search


def findGender(urlLink):
    gender = ''
    ## put a url where the desciption of the author is avilable
    
    url = urlLink
   
    htmlText = requests.get(url).text
    #print(htmlText)
    soup = BeautifulSoup(htmlText, 'html.parser')
    
    
    # gathers all the p tags from the html
    soupstring=str(soup.find_all("div")) + str(soup.find_all("p"))
    #print(soupstring)
    shetags = [" she.", " she, "," she ", " She ","She ", 
              " her.", " her, "," her "," Her ","Her "
              " her."," hers, "," hers ","Hers "," Hers"
               , "She’s ", " she's " ]
    
    hetags = [" he.", " he, "," he ", " He ","He ", 
              " his.", " his, "," his "," His ","His "
              " him."," him, "," him "," Him ", "He’s ", " he's " ]
    
    theytags = [" they ", "They ", " them ", "Them " ]

    ##checks if male or female pronouns are present in the desciption

    containsfemale = any(femalepronouns in soupstring for femalepronouns in shetags)
    containsmale = any(malepronouns in soupstring for malepronouns in hetags)
    containsthey = any(theypronouns in soupstring for theypronouns in theytags)

    if containsfemale:
        gender = "female"
    elif containsmale:
        
        gender = "male"
#     elif containsthey:
#         gender = "they/them"
    else:
        gender = "inconclusive"
    print(gender) 



In [25]:
findGender("https://www.cmu.edu/energy/people/alpha-list/zico-kolter.html")

male


In [6]:
# returns whether or not a substring exists in a string
def substring_exists(string, sub_string):
    return string.find(sub_string)


In [7]:
# returns a list of the first-authors
def getFirstAuthorList():
    for column in data[['first-author']]:
        authors = data[column]

        '''
        Prints out column name and its values
        '''
        #print('Column Name: '+column)
        # print(authors.values)

    lst_authors = authors.values
    return lst_authors


In [53]:
# As you can see, we need to look into optimizing the findGender function
# I will continue to work on the googlesearch so that it gives the best websites to navigate
print(data[['first-gender']])


   first-gender
0              
1              
2              
3              
4              
5              
6              
7              
8              
9              
10             
11             
12             
13             
14             
15             
16             
17             
18             


In [19]:
def createKeyValue():
    #creates a dictionary for key value pairs
    linkDictionary = {}
    counter = 0
    linkArray = []
    lst = getFirstAuthorList()
    #if we have any particular domains which we dont want to include in out searches we can input them in this list
    unwantedLinks = ["pdf", "dblp"]
    for person in lst:

        query = person+" " + data['first-author-affiliation'].values[counter]+" .edu"

        for link in search(query, tld="co.in", num=10, stop=10):
            # appends .edu and .org links into the array
            if(substring_exists(link, ".edu") != -1 or substring_exists(link, ".org") != -1 ):
                #checks if each link contains anything from the list of unwanted list
                containsUnwanted = any(elements in link for elements in unwantedLinks)
                if(containsUnwanted == False):
                    linkArray.append(link)

        # creates the pair in the dictonary
        linkDictionary[person] = linkArray
        linkArray = []
        counter += 1

    return linkDictionary

In [20]:
createKeyValue()

{'Surya Ganguli': ['https://ganguli-gang.stanford.edu/',
  'https://ganguli-gang.stanford.edu/people.html',
  'https://ganguli-gang.stanford.edu/pubs.html',
  'https://ganguli-gang.stanford.edu/teaching.html',
  'https://profiles.stanford.edu/surya-ganguli',
  'https://ganguli-gang.stanford.edu/surya.html',
  'https://en.wikipedia.org/wiki/Surya_Ganguli',
  'https://www.simonsfoundation.org/people/surya-ganguli/'],
 'Hal Daumé III': ['http://www.umiacs.umd.edu/~hal/',
  'https://www.cs.umd.edu/people/hal3'],
 'Akshat Kumar': ['https://scholarworks.umass.edu/open_access_dissertations/734/',
  'https://www.cics.umass.edu/news/akshat-kumar-wins-icaps-2014-best-dissertation-award',
  'http://www.mysmu.edu/faculty/akshatkumar/',
  'https://www.aaai.org/ocs/index.php/ICAPS/ICAPS16/rt/bio/13124/0'],
 'Alan Fern': ['https://eecs.oregonstate.edu/people/fern-alan',
  'https://web.engr.oregonstate.edu/~afern/',
  'https://eecs.oregonstate.edu/people/ferna-profile',
  'http://web.engr.oregonstate.

In [ ]:
## im currently working on the loop to call findGender 